See [circular_exp_fitting](circular_exp_fitting.ipynb)  and  [sersic_integral_experiments](sersic_integral_experiments.ipynb) for prior art

In [1]:
# needed on macs due to subtle multiprocessing differences used in dynesty
import sys
if sys.platform == 'darwin':
    import multiprocessing
    multiprocessing.set_start_method('fork')

In [2]:
import math

import numpy as np
np.seterr(invalid='ignore')

from scipy import special

import sympy

import regions
from regions import CirclePixelRegion, PixCoord, RectanglePixelRegion, CirclePixelRegion, PolygonPixelRegion

In [3]:
from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
TWOPI = 2*np.pi

# How to update integrals for elliptical fitting

intruce a circularity parameter $c \equiv 1-e = b/a$

In [5]:
r,θ,c = sympy.symbols('r,θ,c', real=True)

In [ ]:
x = r*c*sympy.cos(θ)
y = r*sympy.sin(θ)
J = sympy.Matrix([x,y]).jacobian([r,θ])
J

In [ ]:
J.det()

In [ ]:
J.det().simplify()

This means all the radial integrals just need a $c$ added - conveniently this will always be withj the $2 \pi$ so that's an easy way to remember where it goes. 

In [9]:
ICOEFFS = [2, -1/3, 4/405, 46/25515, 131/1148175, -2194697/30690717750]

def f_bn(n):
    """
    Ciotti and Bertin 99, valid for n >~ 0.36
    """
    return np.sum([C*n**(1-i) for i, C in enumerate(ICOEFFS)], axis=0)

def sersic_profile(R, Ie, Re, n):
    bn = f_bn(n)
    return Ie * np.exp(-bn*((R/Re)**(1/n)-1))

def log_sersic_profile(R, Ie, Re, n):
    bn = f_bn(n)
    return np.log(Ie) - bn*((R/Re)**(1/n)-1)

Now implement technique 5 for the sersic integral from sersic_integral_experiments, but with the added factor of c

In [10]:
def make_grid_Iweighted(nr, nphi, outerr, Re, n):
    bn_val = f_bn(n)

    gamma2 = special.gammainc(2*n, outerr**(1/n) * Re**(-1/n) * bn_val)

    u = np.linspace(0, 1, nr)
    r = Re * (special.gammaincinv(2*n, u * gamma2) / bn_val)**n
    phi = np.linspace(0, 2*np.pi, nphi+1)[:-1]

    return np.meshgrid(r, phi)

def sersic_outer_integral(Router, Re, n, e):
    bnval = f_bn(n)
    numer1 = 4 * np.pi * Re * Re * bnval**(-2*n) * n*n * np.exp(bnval) * (1-e)
    
    # non-trivial thing here: the scipy gammainc is a *regularized* gamma function meaning it has a factor of 1/gamma(s), 
    # so we square our prefactor to have scipy's gammainc match the sympy definition which is not regularized
    numer2 = special.gamma(2*n)**2 * special.gammainc(2*n, Router**(1/n) * Re**(-1/n) * bnval)
    
    denom = special.gamma(2*n + 1)

    return numer1 * numer2 / denom

def sersic_integral(Re, n, e, pa, region, x0=0, y0=0, nr=100, nphi=100):
    """
    Note the the region, Re, x0, and y0 should all be in pixels
    """

    outerr = np.max(np.abs(region.bounding_box.extent))
    outer_integ = sersic_outer_integral(outerr, Re, n, e)

    r, phi = make_grid_Iweighted(nr, nphi, outerr, Re, n)
    xg_unrotated = r*(1-e)*np.cos(phi) + x0
    yg_unrotated = r*np.sin(phi) + y0
    c = np.cos(pa)
    s = np.sin(pa)
    xg = xg_unrotated * c + yg_unrotated * s
    yg = xg_unrotated *-s + yg_unrotated * c
    msk = region.contains(regions.PixCoord(x=xg, y=yg))

    frac_in_grid = np.sum(msk)/msk.size

    return outer_integ * frac_in_grid

Now define the generator functions:

In [11]:
def get_sersic_r(npoints, outerr, Re, n, rstate=np.random.default_rng()):
    bn_val = f_bn(n)

    gammaparam = 2*n

    gamma2 = special.gammainc(gammaparam, outerr**(1/n) * Re**(-1/n) * bn_val)

    u = rstate.uniform(size=npoints)
    r = Re * (special.gammaincinv(gammaparam, u * gamma2) / bn_val)**n

    return r

def generate_unif_box(n=None, halfsize=1, density=None, rstate=np.random.RandomState()):
    if n is not None and density is not None:
        raise ValueError('cannot give both n and density')
    elif density is not None:
        A = (halfsize*2)**2
        n = density * A
        
    if n != int(n):
        remainder = n - int(n)
        n = int(n)
        if np.random.rand(1)[0] < remainder:
            n += 1
    else:
        n = int(n)
        
    x = rstate.uniform(-halfsize, halfsize, n)
    y = rstate.uniform(-halfsize, halfsize, n)

    return x, y

def uniform_in_region(density, region, rstate=np.random.RandomState()):
    maxpx = np.max(region.bounding_box.extent)
    xs, ys = generate_unif_box(halfsize=maxpx, density=density, rstate=rstate)
    msk = region.contains(PixCoord(xs, ys))

    return xs[msk], ys[msk]
    
def generate_sersic_dglx(n=None, Re=1, nsersic=1.5, e=0, pa=0, density=None, region=None, rstate=np.random.RandomState()):
    if n is not None and density is not None:
        raise ValueError('cannot give both n and density')
    elif density is not None:
        # density is the mean density within the HLR
        hA = np.pi*Re**2
        n = 2 * density * hA
        
    if n != int(n):
        remainder = n - int(n)
        n = int(n)
        if np.random.rand(1)[0] < remainder:
            n += 1
    else:
        n = int(n)
        
    r = get_sersic_r(n, np.inf, Re, nsersic, rstate)
    phi = rstate.uniform(0, 2*np.pi, n)

    x_unrotated = (1-e) * np.cos(phi)*r
    y_unrotated = np.sin(phi)*r

    c = np.cos(pa)
    s = np.sin(pa)
    x = x_unrotated * c + y_unrotated * s
    y = x_unrotated *-s + y_unrotated * c

    if region is not None:
        msk = region.contains(PixCoord(x, y))
        x = x[msk]
        y = y[msk]

    return x, y

def glx_and_uniform(glx_hdensity, bkg_density, region, Re=1, nsersic=1.25, e=0, pa=0, rstate=np.random.RandomState()):
    g = generate_sersic_dglx(density=glx_hdensity, Re=Re, nsersic=nsersic, region=region, rstate=rstate, e=e, pa=pa)
    b = uniform_in_region(bkg_density, region=region, rstate=rstate)

    return np.concatenate((g, b), axis=1), g, b

In [ ]:
test_region = RectanglePixelRegion(PixCoord(0, 0), 10, 10)
_, g, b = glx_and_uniform(100, 5, test_region, Re=1, nsersic=1.25, e=.5, pa=np.pi/3)
plt.scatter(b[0],b[1], s=5, alpha=.5)
plt.scatter(g[0],g[1], s=5, alpha=.5, c='r')
len(g[0]), len(b[0])

In [13]:
circular_large_reg = CirclePixelRegion(PixCoord(0, 0), 10)
circular_small_reg = CirclePixelRegion(PixCoord(0, 0), 3)
rectangular_large_reg = RectanglePixelRegion(PixCoord(0, 0), 20, 20)
rectangular_small_reg = RectanglePixelRegion(PixCoord(0, 0), 6, 6)

offset_region1 = RectanglePixelRegion(PixCoord(2, 3), 2, 3)
offset_region2 = RectanglePixelRegion(PixCoord(0, -1), 6, 6)
offset_regions = offset_region1 | offset_region2

offset_poly_region = PolygonPixelRegion(PixCoord([-3, -3, 3, 3, 1, 1, -3], [2, -4, -4, 5, 5, 3, 2]))

all_regions = {nm: globals()[nm] for nm in 'circular_large_reg,circular_small_reg,rectangular_large_reg,rectangular_small_reg,offset_poly_region'.split(',')}

# Set up fitting code

`region` is a fixed region - if None the integrals are implied to infity.

Parameters:
* x0, y0 -> center
* Re -> half-light radius
* n -> sersic parameter
* fbkg -> background fraction of all the stars

In [14]:
import dynesty
from dynesty import plotting as dyplot

param_names = 'x0, y0, Re, n, e, padeg, fbkg'.split(', ')

Derive the bounded scale-free prior generator

In [ ]:
σ, x, a, b, A, F = sympy.symbols('σ,x,a,b,A,F', real=True, positive=True)
Asoln = sympy.solve(sympy.integrate(A/σ,(σ,a,b)) - 1, A)[0]
sympy.solve(sympy.integrate(A/σ,(σ,a,x)).subs(A, Asoln) - F, x)[0]

Prior below is that x0, y0 are on U[-1, 1], Re is scale-free on [.1, 5], n is U[1, 1.5], fbkg is U[.1, .9]

In [ ]:
test_region = circular_large_reg
test_data, tg, tb = glx_and_uniform(50, 2, test_region, nsersic=1.25, Re=2, e=.5, pa=math.radians(30))
truths = [0, 0, 2, 1.25, .5, 30, -1]

truths[-1] = len(tb[0])/(len(tb[0]) + len(tg[0]))

plt.scatter(*tg,s=1)
plt.scatter(*tb,s=2)

len(tg[0]), len(tb[0]), truths[-1]

In [17]:
uppri = 5
lpri = .1

def prior_transform(u):
    x = np.empty_like(u)
    x[:2] = u[:2]*2 -1
    x[2] = (lpri**(1-u[2]) * uppri**u[2])
    x[3] = u[3]*.5 + 1.
    x[4] = u[4]*.9
    x[5] = u[5]*180
    x[6] = u[6]*.9

    return x

In [18]:
LTWOPI = np.log(2*np.pi)

def loglike(p, data, region):
    x0, y0, Re, n, e, padeg, fbkg = p
    pa = math.radians(padeg)
    cparam = 1 - e
    x, y = data

    dx = x-x0
    dy = y-y0
    c = math.cos(pa)
    s = math.sin(pa)
    x_rotated = dx * c + dy *-s
    y_rotated = dx * s + dy * c


    r = np.hypot(x_rotated/cparam, y_rotated)
    lgal = np.log(1-fbkg) + log_sersic_profile(r, 1, Re, n) - np.log(sersic_integral(Re, n, region=region, x0=x0, y0=y0, nr=500, nphi=100, e=e, pa=pa))
    lbkg = np.log(fbkg/region.area)
    ll = np.logaddexp(lgal, lbkg)

    return np.sum(ll)
    

In [ ]:
with dynesty.pool.Pool(12, loglike, prior_transform,
                                logl_kwargs={'data': test_data, 'region':test_region},
                      ) as pool:
    dsampler = dynesty.DynamicNestedSampler(pool.loglike, pool.prior_transform, pool=pool,
                                            ndim=len(param_names),sample='rslice',
                                            periodic=[5])
    dsampler.run_nested(maxiter=100000,  wt_kwargs={'pfrac': 1.0})

In [ ]:
fig, axes = dyplot.runplot(dsampler.results)

span = [1,1, .95, 1, 1, 1, 1]

fig, axes = dyplot.traceplot(dsampler.results, labels=param_names, truths=truths,span=span)
fig, axes = dyplot.cornerplot(dsampler.results, labels=param_names, truths=truths,span=span)
fig, axes = dyplot.cornerpoints(dsampler.results, labels=param_names, truths=truths)